In [35]:
#import libraries
import pyodbc 
import json


In [36]:
#connect to db
server_name = "INTERN-LAPTOP\MSSQLSERVER02"
database_name = "iabcategoriesdb"

connection_string = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes;TrustServerCertificate=yes'

connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

In [ ]:
#create iab_categories table
create_iabCategories_table_query = """
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'iabCategories')
BEGIN
    CREATE TABLE iabCategories (
        category_id INT PRIMARY KEY IDENTITY(1,1),
        category_code NVARCHAR(255) NOT NULL UNIQUE,
        category_name NVARCHAR(255) NOT NULL
    )
END
"""
cursor.execute(create_iabCategories_table_query)

In [ ]:
#create keywords table
create_keywords_table_query = """
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'keywords')
BEGIN
    CREATE TABLE keywords (
        keyword_id INT PRIMARY KEY IDENTITY(1,1),
        keyword NVARCHAR(255) NOT NULL,
        keyword_language NVARCHAR(255) NOT NULL,
        UNIQUE (keyword, keyword_language)
    )
END
"""

cursor.execute(create_keywords_table_query)

In [ ]:
#create iabCategories_keywords table
create_iabCategories_keywords_table_query = """
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'iabCategories_keywords')
BEGIN
    CREATE TABLE iabCategories_keywords (
        category_id INT,
        keyword_id INT,
        FOREIGN KEY (category_id) REFERENCES iabCategories(category_id),
        FOREIGN KEY (keyword_id) REFERENCES keywords(keyword_id)
    )
END
"""
cursor.execute(create_iabCategories_keywords_table_query)

In [ ]:
#insert data in iabCategories table
insert_iab_categories_data_query = """
INSERT INTO iabCategories (category_code, category_name)
VALUES (?, ?)
"""

with open("../dbHandler/iab_categories_initial_data.json", "r") as file:
    data = json.load(file)

for key, value in data["iabCategories"].items():
    try:
        category_name = value['name']


        insert_iab_categories_data_to_insert = (f"{key}", f"{category_name}")
        
        cursor.execute(insert_iab_categories_data_query, insert_iab_categories_data_to_insert)
    except Exception as e:
        print(f"The category code '{key}' already exist in the database")
connection.commit()

In [41]:
#insert data in keywords table and associate tables
insert_keywords_data_query = """
INSERT INTO keywords (keyword, keyword_language)
VALUES (?, ?)
"""

inser_iabCategories_keywords_query = """
INSERT INTO iabCategories_keywords (category_id, keyword_id)
VALUES (?, ?)
"""

find_id_with_category_code_query = """
SELECT category_id FROM iabCategories WHERE category_code = ?
"""

for category_code, category_data in data["iabCategories"].items():
    keywords = category_data["keywords"]
    language = "en"

    cursor.execute(find_id_with_category_code_query, f"{category_code}")
    category_id_to_associate_tuple = cursor.fetchone()
    category_id_to_associate = category_id_to_associate_tuple[0]


    if len(keywords) > 0:
        for keyword in keywords:
            try:
                lowercase_keyword = keyword.lower()
                keywords_data_to_insert = (f"{lowercase_keyword}", language)

                cursor.execute(insert_keywords_data_query, keywords_data_to_insert)
                connection.commit()
                

                cursor.execute("SELECT @@IDENTITY AS last_id")
                result = cursor.fetchone()
                last_keyword_id = result[0]

                iabCategories_keywords_data_to_insert = (category_id_to_associate, last_keyword_id)
                cursor.execute(inser_iabCategories_keywords_query, iabCategories_keywords_data_to_insert)
                connection.commit()
            except Exception as e:
                print(f"The keyword(s) '{keyword}' already exist in the database, in relation to the '{language}' language.")


In [42]:
#close db connection
connection.close()